# BQSKit Intermediate Representation (IR) Tutorial

This tutorial will introduce a user to the core objects in the BQSKit package, as well as, how to manipulate them.

## 1. Unitary Matrices

In BQSKit, most objects are part of the unitary class hierarchy. The base class, `Unitary`, represents a map from zero or more real numbers to a unitary matrix. All `Unitary` objects have a `get_unitary` method which takes in the unitaries parameters and returns the concrete `UnitaryMatrix` object.

In the next section, we will learn more about gates, but for now we will use two well known gates to learn about `Unitary` objects.

In [1]:
from bqskit.ir.gates import CNOTGate, RZGate

cnot = CNOTGate() # The constant Controlled-Not gate
rz = RZGate() # Z-Rotation gate that takes one angle as a parameter

In this first exercise, let's ensure these gates are part of the unitary class hierarchy.

In [2]:
from bqskit.qis.unitary import Unitary

assert isinstance(cnot, Unitary)
assert isinstance(rz, Unitary)

All `Unitary` objects contain a `get_unitary` method that takes zero or more parameters and returns a `UnitaryMatrix`. In this excersie, we will calculate the matrix for both gates. Try changing the parameters, do you get what you expect to get?

In [5]:
cnot_utry = cnot.get_unitary()
rz_utry = rz.get_unitary([0])

print("CNOT's Unitary")
print(cnot_utry)
print()

print("RZ's Unitary")
print(rz_utry)

CNOT's Unitary
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]

RZ's Unitary
[[0.98877108-0.14943813j 0.        +0.j        ]
 [0.        +0.j         0.98877108+0.14943813j]]


We can ask the `Unitary` for the number of parameters in takes in its `get_unitary` method.

In [6]:
assert cnot.num_params == 0
assert rz.num_params == 1

# The cnot gate is constant since it takes zero parameters
assert cnot.is_constant()

# The rz gate is not constant and hence parameterized since it takes one or more parameters
assert rz.is_parameterized()

`Unitary` objects in BQSKit support arbitrary qudits rather than just qubits. Each `Unitary` has a `get_radixes` method that returns a list containing the base for each qudit the Unitary acts on. Try creating another mixed-base `Unitary` in the following code.

In [15]:
from bqskit.qis.unitary import UnitaryMatrix

# Generate a random unitary sampled from the Haar random distribituion
utry = UnitaryMatrix.random(4)
assert utry.num_qudits == 4
assert utry.is_qubit_only()

# The random generation also takes an optional parameter `radixes`.
utry_qutrit = UnitaryMatrix.random(4, [3, 3, 3, 3])
assert utry_qutrit.num_qudits == 4
assert utry_qutrit.is_qutrit_only()

utry_mixed = UnitaryMatrix.random(3, [2, 3, 4])
assert utry_mixed.num_qudits == 3
assert utry_mixed.radixes == (2, 3, 4)

`UnitaryMatrix` objects are concrete matrices that are also constant `Unitary` objects. They support most of the numpy api and for the most part can be used like normal numpy arrays. If you perform an operation that is closed for unitaries (conjugate, transpose, matmul, ...) the resulting object will remain a `UnitaryMatrix` object, otherwise it will downgrade to a `np.ndarray`.

In [18]:
assert isinstance(utry, Unitary)

import numpy as np
utry1 = UnitaryMatrix.random(2)
utry2 = UnitaryMatrix.random(2)
utry3 = utry1 @ utry2
utry4 = np.kron(utry3, UnitaryMatrix.identity(1))
print(utry4)
assert isinstance(utry4, UnitaryMatrix)

[[ 0.48631694-0.48737702j -0.51316014+0.09027776j -0.40648524+0.19054718j
   0.22377464-0.05350239j]
 [-0.26484955+0.33631281j -0.65019141+0.42901655j -0.08600916-0.15602559j
  -0.31121789+0.28521255j]
 [-0.2600829 +0.5236309j  -0.1237414 -0.11892378j -0.3255275 +0.48157631j
   0.37610165-0.38649246j]
 [ 0.01204634-0.0270246j   0.266184  -0.14630038j -0.53729726+0.37436309j
  -0.66588016+0.18610312j]]
(-2.7800190357347017+1.5213940441512723j)


AssertionError: 